## 5.8 다항 베이지안 multinomial Bayesian

**다항모델 Multinomial Bayesian**은 앞서 이항분포가 사건의 발생여부에 따른 이진적이었다면, 다항분포는 단어의 발생빈도에 따라 문서를 분류하는 문제를 예로 들 수 있다. 단어가 1번 발생할 확률, 2번 발생할 확률을 계산하게 된다.


###  계산

속성의 집합 x = (x1,...,xn), 그리고 Ck (k개의 클래스), 추론은 최대의 사후확률 $p(C_k | x_1,\ldots,x_n)$로 결정된다.

$posterior = \frac{prior \times likelihood}{evidence}$

$P(C_k \vert x) = \frac{P(C_k) \ P(x \vert C_k)}{P(x)}$

$
\begin{align}
P(C_k \vert x_1, \dots, x_n)
    & \varpropto P(C_k, x_1, \dots, x_n) \\
    & \varpropto P(C_k) P(x_1 \vert C_k) \
         P(x_2\vert C_k) P(x_3\vert C_k) \cdots \\
    & \varpropto P(C_k) \prod_{i=1}^n P(x_i \vert C_k)
\end{align}
$

파이처럼 보이는것? product

### 예제

문서가 다음가 같이 주어졌다고 하자.
문서는:
* **문장**으로 분리한다. 이 경우 문장을 나누는 기준이 단순히 '.'으로 분리하기 쉽지 않을 수 있다. Mr. 같은 약어에 점을 포함되거나, 의문부호로 문장을 끝내거나, 마침표로 끝내지 않는 경우도 있기 때문이다.
* **단어**로 분리한다. 단어로 분리되면, Bag of Words 모델에 따라, 단어의 앞 뒤에 무엇이 왔는지 무시되고, 오로지 단어의 집합으로 구성된다.
* **품사**도 식별할 필요가 있다. 한국어는 조사가 붙어 격변화를 하고, 형용사도 다양하게 변화하므로 어근을 추출해 사용하기도 한다.
* **불용어**는 제거해준다.

이렇게 추출된 단어로 word vector를 생성한다. word vector는 (단어 key, 값 value)로 구성되면, 이 경우 값은 단어존재, 단어빈도, TF-IDF 등이 사용된다.

베이지안과 같은 지도기계학습을 적용하기 위해서는, 각 문서에 대해 분류가 선행되어야 한다. 이를 위해서 별도로 판정 작업이 필요하다.
문서 1, 3, 4, 6, 8, 9는 부정적, 반면에 문서2, 5, 7, 10 4건은 긍정적이다.
부정의 label은 0, 긍정의 label은 1로 하자.

문서번호 | 문서 | 단어 | 클래스
-----|------|------|------
1 | I am sorry he has fleas poor dog | 'sorry','fleas','poor','dog' | 1
2 | take the dog to the park my dog would love it | 'dog','park','dog','love' | 0
3 | quit posting stupid worthless garbage | 'quit','posting','stupid','worthless','garbage' | 1
4 | my dog has fleas quit buying worthless dog food stupid' | 'dog','fleas','quit','buying','worthless','dog','food','stupid' | 1
5 | dog is so cute I love him | 'dog','cute','love' | 0
6 | 강아지 벼룩 미안해 불쌍해 | '강아지','벼룩','미안해','불쌍해' | 1
7 | 강아지 공원 강아지 좋아해 | '강아지','공원','강아지','좋아해' | 0
8 | 멍청하게 쓸데없는 쓰레기 포스팅 | '멍청하게','쓸데없는','쓰레기','포스팅' | 1
9 | 강아지 벼룩 쓸데없는 강아지 음식 사지마 멍청하게 | '강아지','벼룩','쓸데없는','강아지','음식','사지마','멍청하게' | 1
10 | 강아지 귀여워 좋아해 | '강아지','귀여워','좋아해' | 0
11 | my love my dog has fleas poor dog | 'dog','dog','fleas','love','poor' | ?
12 | 강아지 벼룩 멍청하게 | '강아지','벼룩', '멍청하게' | ?


#### 사전확률

* 총문서 10건
    * 문서(0)은 4
    * 문서(1)은 6

* P(1) prior = $\frac{N_1}{N} = \frac{6}{10} = 0.6$ 총 10개 문서 중 6개
* P(0) prior = $\frac{N_0}{N} = \frac{4}{10} = 0.4$ 총 10개 문서 중 4개

#### 조건확률

* 전체 단어빈도 46 (영어 24 + 한글 22)
    * 긍정(0)일때 전체 단어빈도 14 (중복제거 8: 'dog','park','love','cute','강아지','공원','좋아해','귀여워')
    * 부정(1)일때 전체 단어빈도 32 (중복제거 21: 'sorry','fleas','poor','dog','quit','posting','stupid','worthless','garbage' ,'buying','food','강아지','벼룩','미안해','불쌍해','멍청하게','쓸데없는','쓰레기','포스팅','음식','사지마')
* 전체 단어빈도 (중복제거) 27 (영어: 4 + 2 + 5 + 2 + 1, 한글: 4 + 2 + 4 + 2 + 1 = 14 + 13)

'sorry'의 조건확률
* P(sorry|1) = $\frac{N_{sorry|1} + 1}{N_1 + N_{voca}} = \frac{1+1}{32+27} = 0.03389831$

'dog'의 조건확률이다. 이 단어는 양 쪽에 모두 3회 나타난다.
* P(dog|0) = $\frac{N_{dog|0} + 1}{N_1 + N_{voca}} = \frac{3+1}{14+27} = 0.09756098$
* P(dog|1) = $\frac{N_{dog|1} + 1}{N_1 + N_{voca}} = \frac{3+1}{32+27} = 0.06779661$

'쓸데없는' 단어의 조건확률이다.
* P(쓸데없는|0) = $\frac{N_{쓸데없는|0} + 1}{N_1 + N_{voca}} = \frac{0+1}{14+27} = 0.02439024$
* P(쓸데없는|1) = $\frac{N_{쓸데없는|1} + 1}{N_1 + N_{voca}} = \frac{2+1}{32+27} = 0.05084746$

#### 예측

dog 2회, fleas, love, poor를 예측하면
* post1 = prior1 x likelihood1 = prior1 x p(dog|1) x p(dog|1) x p(fleas|1) x p(poor|0) x p(love|0)
* post0 = prior1 x likelihood1 = prior0 x p(dog|0) x p(dog|0) x p(fleas|0) x p(poor|0) x p(love|0)

이 계산에 필요한 조건부 확률은:
* p(dog|0) = 0.09756098
* p(fleas|0) = 0.02439024
* p(poor|0) = 0.02439024
* p(love|0) = 0.07317073

* p(dog|1) = 0.06779661
* p(fleas|1) = 0.05084746
* p(poor|1) = 0.03389831
* p(love|1) = 0.01694915

In [1]:
import math
# dog^2 * fleas * poor * love
print("post0: {:.16f}".format(0.4*math.pow(0.09756098,2)*0.02439024*0.02439024*0.07317073))
# dog^2 * fleas * poor * love
print("post1: {:.16f}".format(0.6*math.pow(0.06779661,2)*0.05084746*0.03389831*0.01694915))

post0: 0.0000001657226369
post1: 0.0000000805679737


0일 경우의 사후확률 post0: 0.0000001657226369은 1일 경우의 사후확률 post1: 0.0000000805679737보다 크다. 그러므로 0으로 결정하게 된다.

### 데이터

앞서 구성한 텍스트 데이터를 가지고, d1, ..., d10을 구성한다.
d1, d3, d4, d6, d8, d9는 부정적이고 d2, d5, d7, d10 4건은 긍정적이다.

In [2]:
postingList=[['sorry','fleas','poor','dog'],
             ['dog','park','dog','love'],
             ['quit','posting','stupid','worthless','garbage'],
             ['dog','fleas','quit','buying','worthless','dog','food','stupid'],
             ['dog','cute','love'],
             ['강아지','벼룩','미안해','불쌍해'],
             ['강아지','공원','강아지','좋아해'],
             ['멍청하게','쓸데없는','쓰레기','포스팅'],
             ['강아지','벼룩','쓸데없는','강아지','음식','사지마','멍청하게'],
             ['강아지','귀여워','좋아해']]
classVec = [1,0,1,1,0,1,0,1,1,0]

### (1) 단어목록

#### 단어의 목록은 집합으로

단어목록은 set로 선언하여, 중복이 없는 단어들을 저장한다.

In [3]:
vocabSet=set([])

add() 함수로 set에 단어들을 저장할 수 있다.

In [4]:
vocabSet.add("Seoul")
f"vacabSet: {vocabSet}"

"vacabSet: {'Seoul'}"

그러나 배열과 다른 특징이 있다. set은 중복을 허용하지 않는다. 동일한 문자열을 또 입력해도, 하나만 보관된다.

In [5]:
vocabSet.add("Seoul")
f"vocabSet: {vocabSet}"

"vocabSet: {'Seoul'}"

다시 깨끗하게 비우고 시작하자.

In [6]:
vocabSet.clear()
f"vocabSet: {vocabSet}"

'vocabSet: set()'

#### 합집합
문서로부터 단어를 하나씩 추가하는데, 이미 있는 단어가 등장하는 경우가 흔하다.
따라서 중복을 막기 위해, union 연산 "|"으로 합집합을 구하게 된다.

In [7]:
citiesA = set(["Seoul","Sydney","Tokyo"])
citiesB = set(["Seoul","New York"])
citiesAll = citiesA|citiesB #A,B합집합 -> 중복제거됨
print(citiesAll)

{'Tokyo', 'Sydney', 'Seoul', 'New York'}


한 줄씩 단어를 가져와 set()로 중복이 없도록 골라내고, 현재의 vacabSet에 더해가면서 중복이 없는 단어의 목록으로 구성한다.

In [8]:
for doc in postingList:
    #vocaSet = vocaSet.union(set(doc))
    vocabSet=vocabSet | set(doc)

중복이 없는, 단어의 갯수와 목록을 출력할 수 있다.

In [9]:
print ("n of vocab: {0}\nvoca: {1}\n".format(len(vocabSet),vocabSet))

n of vocab: 27
voca: {'강아지', '음식', '쓰레기', 'park', '쓸데없는', 'dog', '미안해', 'sorry', '벼룩', 'food', 'garbage', '좋아해', 'stupid', '귀여워', 'posting', 'poor', 'worthless', 'cute', '공원', '불쌍해', '포스팅', '사지마', 'quit', 'fleas', 'love', '멍청하게', 'buying'}



### (2) 단어 벡터

단어벡터 Word Vector는 단어빈도로 구성된 배열을 말한다.
이를 구성하기 위해서는:
* **중복되지 않는 단어목록**을 구하고, 몇 개의 단어인지 알아낸다.
* 단어의 수만큼 단어벡터를 만들고, **초기화**한다.
* 문서의 단어벡터에 **단어빈도**를 입력한다.
* 모든 문장에 단어별 빈도를 저장한 **단어벡터**를 생성한다.

#### 단어벡터 크기 결정해서 초기화

vocabSet는 중복되지 않는 단어목록을 가지고 있다.
set은 리스트와 달리 순서가 없는 특징이 있으므로 **indexing**기능을 사용할 수 없다.
우리는 단어의 indexing이 필요하므로 set을 리스트로 변환해서 사용하자.

In [10]:
vocabList=list(vocabSet)

그 수만큼 단어벡터의 크기를 정하자.
0으로 초기화한다.

In [11]:
#wordVec = [0]*len(vocabList)
wordVec = [0]*len(vocabSet)

출력해보면, 앞서 찾아낸 중복없는 단어의 갯수만큼 word vector가 설정된다.

In [12]:
 print("wordVec: {}".format(wordVec))

wordVec: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### 한 문장을 단어벡터로 만들어 보기

문장은 이미 단어로 분리되어 있다. 단어별로 초기화된 단어벡터에 빈도를 입력해보자.
빈도는 단순히 1로 하거나, 발생빈도를 그대로 적을 수 있다.
**단어목록에 없는 경우**에는 문제가 된다. 앞서 단어목록을 만들면서 빼먹은 경우인데, 새로 문장이 등장하면 그런 경우가 발생할 수 있다.

In [13]:
for word in postingList[1]:
    if word in vocabList:
        print ("...inserting '{}'".format(word))
        #returnVec[vocabList.index(word)] = 1 # 0 if none, 1 if exists
        wordVec[vocabList.index(word)] += 1 # num of frequencies
    else:
        print ("the word: %s is not in my Vocabulary!".format(word))

...inserting 'dog'
...inserting 'park'
...inserting 'dog'
...inserting 'love'


위 단어벡터의 단어 순서대로 빈도가 적혀졌다. 발생한 빈도가 적혀지니까, 2회 발생한 경우는 2로 1회는 1로 적힌다.

In [14]:
print ("len: {} word vector: {}".format(len(wordVec), wordVec))

len: 27 word vector: [0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


#### 함수로 만들기

단어벡터를 만드는 기능은 모든 문서에 대해 실행되어야 한다.
앞서 단어벡터 만드는 작업을 함수로 만들어 실행하자.

In [15]:
# define function setOfWords2Vec
def setOfWords2Vec(vocabList, inputSet):
    wordVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            print ("...inserting '{}'".format(word))
            #wordVec[vocabList.index(word)] = 1 # 0 if none, 1 if exists
            wordVec[vocabList.index(word)] += 1 # num of frequencies
        else: print ("the word: '{}' is not in my Vocabulary!".format(word))
    return wordVec

#### 단어벡터 생성

모든 문장에 대해 단어별 빈도를 저장한 훈련단어벡터를 생성해 보자.

In [16]:
trainMat=[]
for postinDoc in postingList:
    trainMat.append(setOfWords2Vec(vocabList, postinDoc))

...inserting 'sorry'
...inserting 'fleas'
...inserting 'poor'
...inserting 'dog'
...inserting 'dog'
...inserting 'park'
...inserting 'dog'
...inserting 'love'
...inserting 'quit'
...inserting 'posting'
...inserting 'stupid'
...inserting 'worthless'
...inserting 'garbage'
...inserting 'dog'
...inserting 'fleas'
...inserting 'quit'
...inserting 'buying'
...inserting 'worthless'
...inserting 'dog'
...inserting 'food'
...inserting 'stupid'
...inserting 'dog'
...inserting 'cute'
...inserting 'love'
...inserting '강아지'
...inserting '벼룩'
...inserting '미안해'
...inserting '불쌍해'
...inserting '강아지'
...inserting '공원'
...inserting '강아지'
...inserting '좋아해'
...inserting '멍청하게'
...inserting '쓸데없는'
...inserting '쓰레기'
...inserting '포스팅'
...inserting '강아지'
...inserting '벼룩'
...inserting '쓸데없는'
...inserting '강아지'
...inserting '음식'
...inserting '사지마'
...inserting '멍청하게'
...inserting '강아지'
...inserting '귀여워'
...inserting '좋아해'


훈련단어벡터를 출력하면, 문장의 수 10개별로 단어의 수 32개에 대해 빈도를 0, 1로 표시하여 출력하고 있다.

In [17]:
f"word vector: {trainMat}"

'word vector: [[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]'

### (3) 사전확률


In [18]:
# trainNB0
# nword_doc-np.zeros([nword,ndoc])
numTrainDocs=len(trainMat) #6

In [19]:
f"num of Train Docs: {numTrainDocs}" #전체문서의 개수

'num of Train Docs: 10'

In [20]:
classVec.count(1) #class [1,0,1,1,0,1,0,1,1,0] (10문장중 부정 문장의 개수)

6

In [21]:
prior1 = classVec.count(1)/numTrainDocs
prior0 = classVec.count(0)/numTrainDocs

In [22]:
f"prior for 1: {prior1} prior for 0: {prior0}"

'prior for 1: 0.6 prior for 0: 0.4'

### (4) 우도

$\frac{\displaystyle 해당단어빈도 numWords + 1}{\displaystyle 총단어빈도 totalNumWords + 고유단어수 vocab}$


#### 고유단어수

In [23]:
V=len(vocabSet)

In [24]:
f"Num of unique words: {V}"

'Num of unique words: 27'

#### 클래스별 총단어빈도, 해당 단어빈도

분모, 분자로 나누어 계산한다.
분모는 클래스별 총단어빈도를,
분자에는 해당 단어빈도를 계산한다.

In [25]:
import numpy as np

p0Num = np.zeros(V)
p1Num = np.zeros(V)
p0Denom = 0.0
p1Denom = 0.0  # numWords with repetition

우도의 분모, 분자를 구한다.
각 클래스의 전체 단어수와 해당 훈련데이터의 단어빈도를 구한다.

In [26]:
for i in range(numTrainDocs):
    if classVec[i] == 1:
        p1Num += trainMat[i]
        p1Denom += sum(trainMat[i]) #p1num안에 있는것 다 더함
        print ("{}. class:{} 빈도합:{}".format(i,classVec[i],p1Num))
    else:
        p0Num += trainMat[i]
        p0Denom += sum(trainMat[i])
        print ("{}. class:{} 빈도합:{}".format(i,classVec[i],p0Num))
        
#마지막 2개가 빈도합
print("\n\n",p1Denom,p0Denom)

0. class:1 빈도합:[0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0.]
1. class:0 빈도합:[0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0.]
2. class:1 빈도합:[0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 0.]
3. class:1 빈도합:[0. 0. 0. 0. 0. 3. 0. 1. 0. 1. 1. 0. 2. 0. 1. 1. 2. 0. 0. 0. 0. 0. 2. 2.
 0. 0. 1.]
4. class:0 빈도합:[0. 0. 0. 1. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 2. 0. 0.]
5. class:1 빈도합:[1. 0. 0. 0. 0. 3. 1. 1. 1. 1. 1. 0. 2. 0. 1. 1. 2. 0. 0. 1. 0. 0. 2. 2.
 0. 0. 1.]
6. class:0 빈도합:[2. 0. 0. 1. 0. 3. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 2. 0. 0.]
7. class:1 빈도합:[1. 0. 1. 0. 1. 3. 1. 1. 1. 1. 1. 0. 2. 0. 1. 1. 2. 0. 0. 1. 1. 0. 2. 2.
 0. 1. 1.]
8. class:1 빈도합:[3. 1. 1. 0. 2. 3. 1. 1. 2. 1. 1. 0. 2. 0. 1. 1. 2. 0. 0. 1. 1. 1. 2. 2.
 0. 2. 1.]
9. class:0 빈도합:[3. 0. 0. 1. 0. 3. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 2. 0. 0.]


 32.0 14

위 코드에서 각 클래스의 총단어빈도를 계산하였다.
**총단어빈도**는 클래스 0에 14개, 클래스2에 32개이다.
단 이들은 중복이 허용된 숫자이므로, 합한다고 해서 vacabSet에 있는 27과 같지 않다.
분자인 단어빈도를 모두 더하면 당연히 분모인 총단어빈도가 된다.

자, 이제 계산을 해보자.
* 0에 대해, p0Denom은 총단어빈도에 해당하고 14이다.
3번째 값은 단어빈도 1이다. 이를 총단어빈도 14로 나누어 계산한다.
```python
0.07142857 = 1/14 (단어빈도/총단어빈도)
```

* 1에 대해서도 마찬가지이다. 단어빈도 1인경우 총단어빈도 32로 나누어 계산한다.
```python
0.03125 = 1/32 (단어빈도/총단어빈도)
```

In [27]:
p1Vect = p1Num/p1Denom
p0Vect = p0Num/p0Denom

In [28]:
print ("---p0:\np0Num=\n{0}\np0Denom={1}\np0Vect\n{2}".format(p0Num, p0Denom, p0Vect))

---p0:
p0Num=
[3. 0. 0. 1. 0. 3. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 2. 0. 0.]
p0Denom=14.0
p0Vect
[0.21428571 0.         0.         0.07142857 0.         0.21428571
 0.         0.         0.         0.         0.         0.14285714
 0.         0.07142857 0.         0.         0.         0.07142857
 0.07142857 0.         0.         0.         0.         0.
 0.14285714 0.         0.        ]


In [29]:
print ("---p1:\n p1Num=\n{0}\np1Denom={1}\np1Vect\n{2}".format(p1Num, p1Denom, p1Vect))

---p1:
 p1Num=
[3. 1. 1. 0. 2. 3. 1. 1. 2. 1. 1. 0. 2. 0. 1. 1. 2. 0. 0. 1. 1. 1. 2. 2.
 0. 2. 1.]
p1Denom=32.0
p1Vect
[0.09375 0.03125 0.03125 0.      0.0625  0.09375 0.03125 0.03125 0.0625
 0.03125 0.03125 0.      0.0625  0.      0.03125 0.03125 0.0625  0.
 0.      0.03125 0.03125 0.03125 0.0625  0.0625  0.      0.0625  0.03125]


이제 스무딩을 해보자.
스무딩을 하지 않으면 0으로 연산하면서 문제가 될 수 있다.

In [30]:
p1Vect=(p1Num+1)/(p1Denom+V) #V:중복이 되지 않는 단어의 수
p0Vect=(p0Num+1)/(p0Denom+V)

In [31]:
print ("---p0:\np0Num=\n{0}\np0Denom={1}\np0Vect\n{2}".format(p0Num+1, p0Denom+V, p0Vect))

---p0:
p0Num=
[4. 1. 1. 2. 1. 4. 1. 1. 1. 1. 1. 3. 1. 2. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1.
 3. 1. 1.]
p0Denom=41.0
p0Vect
[0.09756098 0.02439024 0.02439024 0.04878049 0.02439024 0.09756098
 0.02439024 0.02439024 0.02439024 0.02439024 0.02439024 0.07317073
 0.02439024 0.04878049 0.02439024 0.02439024 0.02439024 0.04878049
 0.04878049 0.02439024 0.02439024 0.02439024 0.02439024 0.02439024
 0.07317073 0.02439024 0.02439024]


In [32]:
print ("---p1:\n p1Num=\n{0}\np1Denom={1}\np1Vect\n{2}".format(p1Num+1, p1Denom+V, p1Vect))

---p1:
 p1Num=
[4. 2. 2. 1. 3. 4. 2. 2. 3. 2. 2. 1. 3. 1. 2. 2. 3. 1. 1. 2. 2. 2. 3. 3.
 1. 3. 2.]
p1Denom=59.0
p1Vect
[0.06779661 0.03389831 0.03389831 0.01694915 0.05084746 0.06779661
 0.03389831 0.03389831 0.05084746 0.03389831 0.03389831 0.01694915
 0.05084746 0.01694915 0.03389831 0.03389831 0.05084746 0.01694915
 0.01694915 0.03389831 0.03389831 0.03389831 0.05084746 0.05084746
 0.01694915 0.05084746 0.03389831]


'dog'은 양쪽에 각 3회씩 등장한다. 우도를 구해보면 각 p(dog|0)=0.09756098, p(dog|1)=0.06779661 이다.
* P(dog|0) = $\frac{N_{dog|0} + 1}{N_1 + N_{voca}} = \frac{3+1}{14+27} = 0.09756098$
* P(dog|1) = $\frac{N_{dog|1} + 1}{N_1 + N_{voca}} = \frac{3+1}{32+27} = 0.06779661$

단어벡터의 2번째 위치를 차지하는 '쓸데없는'은 부정(1)인 경우에만 등장한다.
* P(쓸데없는|0) = $\frac{N_{쓸데없는|0} + 1}{N_1 + N_{voca}} = \frac{0+1}{14+27} = 0.02439024$
* P(쓸데없는|1) = $\frac{N_{쓸데없는|1} + 1}{N_1 + N_{voca}} = \frac{2+1}{32+27} = 0.05084746$

이때 주의해야 할 점은, 인덱스 값을 순서를 찾아서 인덱스로 사용하면 안된다.
인덱스는 Set에서 리스트를 생성하면서 순서가 변경되기 때문이다.
**index() 함수로 인덱스를 찾아서** 사용하자.

In [33]:
p0Vect[vocabList.index('dog')], p1Vect[vocabList.index('dog')] #긍/부

(0.0975609756097561, 0.06779661016949153)

In [34]:
p0Vect[vocabList.index('쓸데없는')], p1Vect[vocabList.index('쓸데없는')] #긍/부

(0.024390243902439025, 0.05084745762711865)

### (5) 예측

"my dog has fleas stupid" 또는 "강아지 벼룩 멍청하게" 이런 한, 영 문장으로 분류를 해보자.

#### 데이터

확률계산을 할 때, 없었던 단어를 제거하면, dog, fleas, love, poor, dog이란 단어가 남는다.

In [35]:
# classifyNB
import math
# my dog has flea problems. Love him, poor dog.
testEntry1 = ['my', 'dog', 'has', 'fleas', 'problems', 'love', 'him', 'poor', 'dog']
#testEntry1 = ['love', 'cute', 'dog']
testData1 = np.array(setOfWords2Vec(vocabList, testEntry1))

the word: 'my' is not in my Vocabulary!
...inserting 'dog'
the word: 'has' is not in my Vocabulary!
...inserting 'fleas'
the word: 'problems' is not in my Vocabulary!
...inserting 'love'
the word: 'him' is not in my Vocabulary!
...inserting 'poor'
...inserting 'dog'


In [36]:
testEntry2 = ['강아지', '불쌍해', '미안해', '어쩌나']
testData2 = np.array(setOfWords2Vec(vocabList, testEntry2))

...inserting '강아지'
...inserting '불쌍해'
...inserting '미안해'
the word: '어쩌나' is not in my Vocabulary!


단어 5개 dog, fleas, love, poor, dog에 대해, 2회 발생한 dog과 나머지 단어에 대해 빈도가 기록된다.

In [37]:
testData1

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0])

In [38]:
testData1[1]

0

한글 단어 3개에 대해 빈도가 기록된다.

In [39]:
testData2

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0])

#### 리스트 연산

In [40]:
import math
post1 = prior1 * \
        math.pow(p1Vect[vocabList.index('dog')],2) * \
        p1Vect[vocabList.index('fleas')] * \
        p1Vect[vocabList.index('poor')] * \
        p1Vect[vocabList.index('love')]

In [41]:
print("post1: {:.10f}".format(post1))

post1: 0.0000000806


In [42]:
import math
post0 = prior0 * \
        math.pow(p0Vect[vocabList.index('dog')],2) * \
        p0Vect[vocabList.index('fleas')] * \
        p0Vect[vocabList.index('poor')] * \
        p0Vect[vocabList.index('love')]

In [43]:
print("post0: {:.10f}".format(post0))

post0: 0.0000001657


사후확률 1이 사후확률 0보다 크다. 따라서 0, 즉 **긍정**으로 예측된다.
앞서 계산한 **가중치를 보면 dog은 긍정단어**로 계산되었다.
부정단어 fleas, poor가 있다고 하더라도, dog과 더불어 love 단어가 사용되어서 0으로 예측된다.

In [44]:
post1>post0

False

In [45]:
np.array([post1,post0]).argmax()

1

#### 벡터연산

#### power

동전의 앞면이 나올 확률은 0.5, 2회 연속 앞면이 나올 확률은 제곱을 해야 하고 0.25가 된다.
```**```는 요소별 연산을 해서, $0.5^2$와 $0.5^1$의 결과 0.25, 0.5가 출력된다.

In [46]:
import numpy as np
np.array([0.5,0.5])**np.array([2,1])

array([0.25, 0.5 ])

```**``` 대신 np.power() 함수를 사용해도 된다.

In [47]:
np.power(np.array([0.5,0.5]),np.array([2,1]))

array([0.25, 0.5 ])

#### product

앞면이 2회 나오는 확률은 제곱을 하고, 이처럼 **거듭되는 확률은 곱셈연산**을 해서 계산한다.
np.prod() 함수는 모든 요소의 곱셈 계산을 한다.

Product은 곱셈을 의미한다. $\prod_{n=2}^4\ n = 2 \times 3 \times 4 = 24$

In [48]:
np.prod(np.power(np.array([0.5,0.5]), np.array([2,1]))) #array([0.25, 0.5 ]) 이것까지 곱해주는것

0.125

사후확률은:
* 사전확률에
* 해당단어의 발생확률에 빈도를 제곱한 후 (```np.power(p1Vect, testData1)```), 모두 곱셈 ```np.prod()```하여 계산한다.

In [49]:
#요소들의 곱을 일일히 해줄 필요 없다! 알아서 해줌
post1=prior1*np.prod(np.power(p1Vect, testData1))
print("post1: {:.16f}".format(post1))

post1: 0.0000000805679707


In [50]:
post0=prior0*np.prod(np.power(p0Vect, testData1))
print("post0: {:.16f}".format(post0))

post0: 0.0000001657226789


In [51]:
post1>post0

False

### (6) sklearn

* X는 배열이나 희소행렬 shape (n_samples, n_features)로 구성
* y는 배열 shape (n_samples,)

#### 데이터

In [52]:
np.array(trainMat).shape #문장 10개에 대하여 중복없이 27단어의 빈도수가 들어가있음

(10, 27)

In [53]:
np.array(classVec).shape

(10,)

#### NB

In [54]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

In [55]:
clf.fit(trainMat, classVec)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### 예측

testData1은 0, testData2는 1로 예측되었다.

In [56]:
print(clf.predict([testData1, testData2])) #영어/한글 test문장

[0 1]


### (7) Spark


In [57]:
import os
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

#### 데이터

데이터는 label, features로 구성하도록 하자.
다음과 같이 단어로 입력해도 좋지만, 여기서는 단어분리와 불용어 제거를 직접 해보자.
한글은 unicode로 설정한다.

In [58]:
df = spark.createDataFrame(
    [
        [1,'I am sorry he has fleas poor dog'],
        [0,'take the dog to the park my dog would love it'],
        [1,'quit posting stupid worthless garbage'],
        [1,'my dog has fleas quit buying worthless dog food stupid'],
        [0,'dog is so cute I love him'],
        [1,u'우리 강아지 벼룩 미안해 불쌍해'],
        [0,u'강아지 공원 강아지 좋아해'],
        [1,u'너 멍청하게 쓸데없는 쓰레기 포스팅'],
        [1,u'강아지 벼룩 쓸데없는 강아지 음식 사지마 멍청하게'],
        [0,u'나 강아지 귀여워 좋아해']
    ],
    ['cls','sent']
)

스키마를 살펴보면, cls는 정수, sent는 문자열로 인식되었다.

In [59]:
df.printSchema()# 스파크 기계학습시 반드시 double로..

root
 |-- cls: long (nullable = true)
 |-- sent: string (nullable = true)



#### StringIndexer
문자열 컬럼을 label index로 변환
우리의 경우는 정수이다. 이 경우는 정수를 문자열로 변환하고 난 후 label index로 변환한다.
빈도수가 높은 문자열이 0이 배정된다.

In [60]:
from pyspark.ml.feature import StringIndexer

labelIndexer = StringIndexer(inputCol="cls", outputCol="label") # double로 만들기 위함

#### RegexTokenizer
텍스트를 분리해서 단어로 만든다.

In [61]:
from pyspark.ml.feature import RegexTokenizer

regexTok = RegexTokenizer(inputCol="sent", outputCol="wordsRegex", pattern="\\s+")
#tokenizer = Tokenizer(inputCol="sent", outputCol="words")

#### 불용어

단어에서 Stopwords를 제거하자.

In [62]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="wordsRegex", outputCol="nostops")

In [63]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

한글 u"나",u"너", u"우리"와 영어 "take"를 제거한다. 영어단어는 불용어 사전을 이용하고, "take"는 앞의 예제와 일치시키기 위해 추가한다.

In [64]:
_mystopwords=[u"나",u"너", u"우리", "take"]
for e in _mystopwords:
    stopwords.append(e)

In [65]:
stop.setStopWords(stopwords)

StopWordsRemover_49256797a507

#### HashingTF
단어 -> word vectors

In [66]:
from pyspark.ml.feature import HashingTF

hashingTF = HashingTF(inputCol="nostops", outputCol="features")

#### Pipeline

In [67]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[labelIndexer,regexTok,stop,hashingTF])

In [68]:
model=pipeline.fit(df)
trainDf = model.transform(df)

In [69]:
trainDf.select("label","nostops","features").show(truncate=False)

+-----+--------------------------------------------------------+------------------------------------------------------------------------------------+
|label|nostops                                                 |features                                                                            |
+-----+--------------------------------------------------------+------------------------------------------------------------------------------------+
|0.0  |[sorry, fleas, poor, dog]                               |(262144,[6155,54556,85735,144961],[1.0,1.0,1.0,1.0])                                |
|1.0  |[dog, park, dog, love]                                  |(262144,[54556,71826,186480],[2.0,1.0,1.0])                                         |
|0.0  |[quit, posting, stupid, worthless, garbage]             |(262144,[1696,67357,111492,186022,247840],[1.0,1.0,1.0,1.0,1.0])                    |
|0.0  |[dog, fleas, quit, buying, worthless, dog, food, stupid]|(262144,[1696,6155,54556,111492,1211

### 훈련데이터

label은 double, features는 vector 타입으로 맞추어 주었는지 확인한다.

In [70]:
trainDf.printSchema()

root
 |-- cls: long (nullable = true)
 |-- sent: string (nullable = true)
 |-- label: double (nullable = false)
 |-- wordsRegex: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nostops: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)



In [71]:
trainDf.select('cls','label','features').show()

+---+-----+--------------------+
|cls|label|            features|
+---+-----+--------------------+
|  1|  0.0|(262144,[6155,545...|
|  0|  1.0|(262144,[54556,71...|
|  1|  0.0|(262144,[1696,673...|
|  1|  0.0|(262144,[1696,615...|
|  0|  1.0|(262144,[23837,54...|
|  1|  0.0|(262144,[5579,587...|
|  0|  1.0|(262144,[1510,587...|
|  1|  0.0|(262144,[81961,17...|
|  1|  0.0|(262144,[3425,587...|
|  0|  1.0|(262144,[58749,64...|
+---+-----+--------------------+



### 모델링
Spark에서는 pyspark.ml.classification의 NaiveBayes 모델을 사용한다.

```python
(featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability', rawPredictionCol='rawPrediction', smoothing=1.0, modelType='multinomial', thresholds=None, weightCol=None)
```
* smoothing 0보다 큰 값을 입력해야 하고, 기본값은 1.0
* modelType은 "multinomial" (기본 값), "bernoulli" 2가지를 지원한다.
TF-IDF와 같은 값은 쓰이면 multinomial을 사용한다.
반면에 존재한다 아니다 0/1로 표시한 경우, bernoulli로 해주자.  
정규분포면 스파크를 못쓰는것!

In [72]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(featuresCol='features', labelCol='label', modelType='multinomial', predictionCol='prediction')

In [73]:
model = nb.fit(trainDf)

클래별 사전확률을 출력해보자.

In [74]:
model.pi

DenseVector([-0.539, -0.8755])

클래스의 조건확률을 출력해보자.

In [75]:
model.theta

DenseMatrix(2, 262144, [-12.4768, -12.4768, -12.4768, -12.4768, -12.4768, -12.4768, -12.4768, -12.4768, ..., -12.4767, -12.4767, -12.4767, -12.4767, -12.4767, -12.4767, -12.4767, -12.4767], 1)

### 예측

데이터가 적어서, train, test로 분리되어 있지 않다.
테스트 데이터로 예측해보려면, 데이터는 DataFrame으로 넘겨주어야 한다. **컬럼명을 알아야** 한다.

In [76]:
predictions=model.transform(trainDf)

In [77]:
predictions

DataFrame[cls: bigint, sent: string, label: double, wordsRegex: array<string>, nostops: array<string>, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [78]:
predictions.select('label','sent','prediction').show()

+-----+----------------------------------+----------+
|label|                              sent|prediction|
+-----+----------------------------------+----------+
|  0.0|              I am sorry he has...|       0.0|
|  1.0|              take the dog to t...|       1.0|
|  0.0|              quit posting stup...|       0.0|
|  0.0|              my dog has fleas ...|       0.0|
|  1.0|              dog is so cute I ...|       1.0|
|  0.0|    우리 강아지 벼룩 미안해 불쌍해|       0.0|
|  1.0|         강아지 공원 강아지 좋아해|       1.0|
|  0.0|너 멍청하게 쓸데없는 쓰레기 포스팅|       0.0|
|  0.0| 강아지 벼룩 쓸데없는 강아지 음...|       0.0|
|  1.0|           나 강아지 귀여워 좋아해|       1.0|
+-----+----------------------------------+----------+



### 평가

BinaryClassificationEvaluator는 두 개의 컬럼 rawPrediction과 label을 서로 비교해서 정확성을 평가한다.

In [79]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")

In [80]:
evaluator.evaluate(predictions)

1.0